# Confision Matrix

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
from deepchecks.vision.dataset import VisionData
from torchvision import models
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
import copy

In [3]:
from deepchecks.vision.checks.performance import ConfusionMatrixReport

## Loading data and model:

In [8]:
from deepchecks.vision.datasets.classification.mnist import load_mnist, MNistNet, mnist_train_loader, mnist_test_loader

ImportError: cannot import name 'load_mnist' from 'deepchecks.vision.datasets.classification.mnist' (/Users/dbi/git/deepchecks/deepchecks/vision/datasets/classification/mnist.py)

In [9]:
def simple_nn():
    torch.manual_seed(42)

    # Define model
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super(NeuralNetwork, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28 * 28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

    model = NeuralNetwork().to('cpu')
    return model

def trained_mnist(simple_nn, mnist_data_loader_train):
    torch.manual_seed(42)
    simple_nn = copy.deepcopy(simple_nn)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(simple_nn.parameters(), lr=1e-3)
    size = len(mnist_data_loader_train.dataset)
    # Training 1 epoch
    simple_nn.train()
    for batch, (X, y) in enumerate(mnist_data_loader_train):
        X, y = X.to('cpu'), y.to('cpu')

        # Compute prediction error
        pred = simple_nn(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return simple_nn

In [10]:
model = trained_mnist(simple_nn(), mnist_train_loader)

NameError: name 'mnist_train_loader' is not defined

In [4]:
train_ds = VisionDataset(mnist_train_loader, label_type='classification')
test_ds = VisionDataset(mnist_test_loader, label_type='classification')

NameError: name 'VisionDataset' is not defined

## Running peformance report on classification

# confusion matrix report object detection

In [4]:
from deepchecks.vision.datasets.detection import coco
import numpy as np

In [5]:
yolo = coco.load_model(pretrained=True)


Using cache found in /Users/dbi/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-1-30 torch 1.10.1 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [6]:
coco_dataloader = coco.load_dataset()

Using downloaded and verified file: /Users/dbi/git/deepchecks/deepchecks/vision/datasets/detection/coco128.zip
Extracting /Users/dbi/git/deepchecks/deepchecks/vision/datasets/detection/coco128.zip to /Users/dbi/git/deepchecks/deepchecks/vision/datasets/detection


In [7]:
from deepchecks.vision.dataset import VisionData

In [8]:
from deepchecks.vision.utils.detection_formatters import DetectionLabelFormatter, DetectionPredictionFormatter
train_ds = VisionData(coco_dataloader, label_transformer=DetectionLabelFormatter(lambda x: x), num_classes=80)

In [ ]:
check = ConfusionMatrixReport(prediction_formatter=DetectionPredictionFormatter(coco.yolo_wrapper))
result = check.run(train_ds, yolo)

/Users/dbi/git/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/Users/dbi/git/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/Users/dbi/git/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling



In [ ]:
result